In [118]:
stellar_out_file = "../stellar/0.gff"

In [119]:
import pandas as pd

stellar_df = pd.read_csv(stellar_out_file, sep='\t', header=None)
stellar_df.columns = ["DNAME", "stellar", "eps-matches", "DBEGIN", "DEND", "PERCID", "DSTRAND", ".", "ATTRIBUTES"]

stellar_df.head()

,DNAME,stellar,eps-matches,DBEGIN,DEND,PERCID,DSTRAND,.,ATTRIBUTES
0,1,Stellar,eps-matches,1024681,1034978,100,+,.,"1;seq2Range=1086931,1097228;cigar=10298M;mutat..."
1,1,Stellar,eps-matches,1036785,1046963,100,+,.,"1;seq2Range=1099235,1109413;cigar=10179M;mutat..."
2,1,Stellar,eps-matches,30950,40845,100,+,.,"1;seq2Range=33150,43045;cigar=9896M;mutations="
3,1,Stellar,eps-matches,784155,793846,100,+,.,"1;seq2Range=831005,840696;cigar=9692M;mutations="
4,1,Stellar,eps-matches,299622,309023,100,+,.,"1;seq2Range=317222,326623;cigar=9402M;mutations="


In [120]:
interesting_attributes = stellar_df['ATTRIBUTES'].str.split(';',expand=True).drop(labels = [0, 2, 3], axis = 1)
interesting_attributes = interesting_attributes[1].str.split('=', expand=True).drop(labels=0, axis = 1)
interesting_attributes = interesting_attributes[1].str.split(',', expand=True)
interesting_attributes.columns = ['QBEGIN', 'QEND']
interesting_attributes.head()

,QBEGIN,QEND
0,1086931,1097228
1,1099235,1109413
2,33150,43045
3,831005,840696
4,317222,326623


In [121]:
stellar_df = stellar_df.drop(labels = ["stellar", "eps-matches", "DSTRAND", "ATTRIBUTES"], axis = 1)

stellar_df = stellar_df.join(interesting_attributes)
stellar_df.head()

,DNAME,DBEGIN,DEND,PERCID,.,QBEGIN,QEND
0,1,1024681,1034978,100,.,1086931,1097228
1,1,1036785,1046963,100,.,1099235,1109413
2,1,30950,40845,100,.,33150,43045
3,1,784155,793846,100,.,831005,840696
4,1,299622,309023,100,.,317222,326623


In [122]:
truth_file = "../ground_truth/0.tsv"

truth_df = pd.read_csv(truth_file, sep='\t')
truth_df.head()

,id,position,length
0,l100-109,851,100
1,l150-101,1299,150
2,l200-124,3796,200
3,l150-97,4970,150
4,l200-43,7405,200


In [123]:
stellar_df["QBEGIN"] = pd.to_numeric(stellar_df["QBEGIN"])
stellar_df["QEND"] = pd.to_numeric(stellar_df["QEND"])

sorted_stellar = stellar_df.sort_values('QBEGIN')
sorted_stellar.head()

,DNAME,DBEGIN,DEND,PERCID,.,QBEGIN,QEND
334,1,1,851,100,.,1,851
438,1,852,1201,100,.,952,1301
168,1,1200,3546,100,.,1450,3796
551,1,355047,355246,100,.,3797,3996
305,1,3547,4520,100,.,3997,4970


In [125]:
len(sorted_stellar["QEND"])

600

In [127]:
truth_df.tail()

,id,position,length
495,l50-108,1086625,50
496,l100-42,1086830,100
497,l100-69,1097227,100
498,l100-8,1099134,100
499,l50-25,1109413,50


In [130]:
sorted_stellar.tail()

,DNAME,DBEGIN,DEND,PERCID,.,QBEGIN,QEND
598,1,1024524,1024680,100,.,1086674,1086830
0,1,1024681,1034978,100,.,1086931,1097228
209,1,1034977,1036788,100,.,1097327,1099138
1,1,1036785,1046963,100,.,1099235,1109413
224,1,1046964,1048576,100,.,1109464,1111076


In [ ]:
sorted_stellar["length"] = 